## BiLSTM에 Attention 적용

In [ ]:
!pip install konlpy
!pip install wandb

In [2]:
import os
import re
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.metrics import F1Score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, Input
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC_online13/notebooks/jiwoong') # 모듈경로 추가

from utils.preprocessing import *
from utils.models import *

import wandb
from wandb.integration.keras.callbacks import WandbMetricsLogger
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 00_jw (jiwoong-team) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC_online13
current_dir = os.getcwd()
csv_dir = os.path.join(current_dir, 'data', 'raw_csv')
train_csv_path = os.path.join(csv_dir, 'merged_train.csv')

/content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC_online13


In [4]:
INPUT_MAX_LENGTH = 350  # 입력 시퀀스 최대 길이
NUM_WORDS = 7000 # 토큰화에 사용할 단어 갯수
EMBEDDING_DIM = 64  # 임베딩 차원

In [5]:
df = pd.read_csv(train_csv_path)
ordered_columns = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화', '일반 대화'] # kaggle에 명시된 순서로 재배치

conversations = df['conversation'].to_list() # input
labels = pd.get_dummies(df['class'])
labels = labels[ordered_columns].to_numpy() # target

In [6]:
# 훈련셋,검증셋 분리
X_train_raw, X_val_raw, y_train, y_val = train_test_split(conversations, labels, test_size=0.2, stratify=labels, random_state=42)

In [7]:
# 토크나이저 생성
tokenizer = create_tokenizer(conversations, NUM_WORDS)
print("생성된 어휘 수:", len(tokenizer.word_index))

생성된 어휘 수: 55065


In [8]:
# 전처리 수행
X_train = preprocessing(X_train_raw, tokenizer, INPUT_MAX_LENGTH)
X_val = preprocessing(X_val_raw, tokenizer, INPUT_MAX_LENGTH)

어텐션 층 추가

In [9]:
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

class AttentionLayer(Layer):
    def __init__(self):
        super(AttentionLayer, self).__init__()

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros")
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        return K.sum(output, axis=1)

In [10]:
def build_model_attention():
  vocab_size = NUM_WORDS + 1  # 패딩

  inputs = Input(shape=(INPUT_MAX_LENGTH,))
  embedding = Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM)(inputs)
  lstm = Bidirectional(LSTM(64, return_sequences=True))(embedding)
  attention = AttentionLayer()(lstm) # 어텐션 추가
  dropout = Dropout(0.5)(attention)
  outputs = Dense(5, activation='softmax')(dropout)
  model = Model(inputs=inputs, outputs=outputs)

  return model

model = build_model_attention()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 350)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 350, 64)        │       448,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 350, 128)       │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_layer                 │ (None, 128)            │           478 │
│ (AttentionLayer)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 515,235 (1.97 MB)

 Trainable params: 515,235 (1.97 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# F1Score 평가 지표
f1 = F1Score(average='micro', name='f1')  # micro f1: 전체 클래스 f1score 평균

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1])

es_callback = EarlyStopping(
    monitor='val_f1',
    patience=3,
    restore_best_weights=True,
    mode='max'
)

mc_callback = ModelCheckpoint(
  f'./notebooks/jiwoong/models/model_attention.keras',
  monitor="val_f1",
  save_best_only=True,
  mode="max"
)

# W&B 프로젝트 초기화
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="jiwoong-team",
    # Set the wandb project where this run will be logged.
    project="DLThon-DKTC",
    # Track hyperparameters and run metadata.
    name=f"jiwoong_BiLSTM_Attention", # 실험 이름
    notes="LLM으로 생성한 일반 대화 데이터를 사용한 베이스라인 모델에 어텐션 적용", # 실험에 대한 간단한 설명
    config={ # 세부 구성 내용
        "experiment_name": "BiLSTM_Attention",
        "general_conversation_type": "LLM에서 생성한 대화 데이터", # 일반 데이터 타입
        "architecture": "BiLSTM(64), Attention",
    },
)

history = model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[es_callback, mc_callback, WandbMetricsLogger()] # WandbMetricsLogger를 콜백으로 넘겨주기만 하면 알아서 epoch별 metric을 기록
)

# 최적의 모델로 검증세트 예측
y_pred = model.predict(X_val)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_val, axis=1)

# wandb Table 생성
report = classification_report(y_true_labels, y_pred_labels, target_names=ordered_columns, output_dict=True)
columns = ["class", "precision", "recall", "f1-score", "support"]
data = []

for label, metrics in report.items():
    if isinstance(metrics, dict):  # dict일 때만 values 가져오기
        row = [label] + [metrics.get(col, None) for col in columns[1:]]
        data.append(row)

table = wandb.Table(columns=columns, data=data)

# wandb에 훈련 결과 기록
wandb.log({
    "classification_report_table": table,
    "macro_f1": report["macro avg"]["f1-score"],
    "accuracy": report["accuracy"],
    "threat_f1": report["협박 대화"]["f1-score"],
    "extortion_f1": report["갈취 대화"]["f1-score"],
    "workplace_bullying_f1": report["직장 내 괴롭힘 대화"]["f1-score"],
    "other bullying": report["기타 괴롭힘 대화"]["f1-score"],
    "general_conversation_f1": report["일반 대화"]["f1-score"],
})

# 실험 종료
run.finish()

Epoch 1/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 12s 36ms/step - accuracy: 0.2063 - f1: 0.2063 - loss: 1.6059 - val_accuracy: 0.3588 - val_f1: 0.3588 - val_loss: 1.3430
Epoch 2/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.4650 - f1: 0.4650 - loss: 1.1994 - val_accuracy: 0.7026 - val_f1: 0.7026 - val_loss: 0.8104
Epoch 3/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.7237 - f1: 0.7237 - loss: 0.7485 - val_accuracy: 0.7349 - val_f1: 0.7349 - val_loss: 0.7003
Epoch 4/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.7899 - f1: 0.7899 - loss: 0.5505 - val_accuracy: 0.8017 - val_f1: 0.8017 - val_loss: 0.5355
Epoch 5/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.8747 - f1: 0.8747 - loss: 0.3941 - val_accuracy: 0.8448 - val_f1: 0.8448 - val_loss: 0.4765
Epoch 6/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - accuracy: 0.9109 - f1: 0.9109 - loss: 0.3003 - val_accuracy: 0.8362 - val_f1: 0.8362 - val_loss: 0.4664
Epoch 7/40
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 30

accuracy,▁
epoch/accuracy,▁▄▆▆▇▇▇█████
epoch/epoch,▁▂▂▃▄▄▅▅▆▇▇█
epoch/f1,▁▄▆▆▇▇▇█████
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▄▃▃▂▂▂▁▁▁▁
epoch/val_accuracy,▁▆▆▇████████
epoch/val_f1,▁▆▆▇████████
epoch/val_loss,█▄▃▂▂▂▁▁▁▂▁▂
extortion_f1,▁
general_conversation_f1,▁


In [12]:
y_pred = model.predict(X_val)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_val, axis=1)

print(confusion_matrix(y_true_labels, y_pred_labels))
print(classification_report(y_true_labels, y_pred_labels, target_names=ordered_columns))

29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step
[[133  18   9  19   0]
 [  7 167   8  11   2]
 [  2   1 183   8   0]
 [  8  17  10 166   1]
 [  0   0   1   0 157]]
              precision    recall  f1-score   support

       협박 대화       0.89      0.74      0.81       179
       갈취 대화       0.82      0.86      0.84       195
 직장 내 괴롭힘 대화       0.87      0.94      0.90       194
   기타 괴롭힘 대화       0.81      0.82      0.82       202
       일반 대화       0.98      0.99      0.99       158

    accuracy                           0.87       928
   macro avg       0.87      0.87      0.87       928
weighted avg       0.87      0.87      0.87       928

